# Imports

In [126]:
# IMPORTS
import numpy as np
from enum import Enum

# CLASS INITIALIZATIONS

In [129]:
class Direction(Enum):
  HORIZONTAL = 1
  VERTICAL = 2

class Coordinate(object):
  def __init__(self, X, Y):
    self.X = X 
    self.Y = Y
  # Primarily for testing to see if Coordinate works
  def __str__(self):
    return "(" + str(self.X) + ", " + str(self.Y) + ")"

class Car(object):
  def __init__(self, letter, coordinate, fuel):
    self.letter = letter  # Remember that A is the most important
    self.coordinates = []
    self.coordinates.append(coordinate)
    self.fuel = fuel

  def isA(self):
    return self.letter == 'A'

  def compareLetters(self, letter):
    return self.letter == letter

  def addCoordinate(self, newCoord):
    self.coordinates.append(newCoord)
    # Update direction
    # L-R (Horizontal)
    if self.coordinates[0].X != newCoord.X and self.coordinates[0].Y == newCoord.Y:
      self.dir = Direction.HORIZONTAL
    # U-D (Vertical)
    else:
      self.dir = Direction.VERTICAL

  def getLetter(self):
    return self.letter; 

  # Primarily for testing to see if Car works
  def __str__(self):
    s = self.letter + " -> ";
    for i in self.coordinates:
      s += str(i)
    return s + " | " + str(self.fuel) + " | " + str(self.dir)

  @staticmethod
  def create(letter, coordinates, fuel):
    return Car(letter, coordinates, fuel)

class Board(object):
  def __init__(self, cars):
    self.size = {'X': 6, 'Y': 6}
    self.cars = cars

  def __str__(self):
    grid = [['.' for i in range(6)] for j in range(6)]
    for car in cars:  
      c = car
      for coord in c.coordinates:
        grid[coord.X][coord.Y] = car.letter
    
    s = ""
    for x in range(6):
      for y in range(6):
        s += grid[x][y]
      s += "\n"
      
    return s

# Extracting Data From Sample-input.txt

In [119]:
demos = []
demoFuels = []

with open('sample-input.txt') as f:
    for line in f:
      if not line.startswith(('#', '\n')):
        in_arr = np.array([line.strip()])
        data = np.char.split(in_arr)[0]
        demos.append(data)
f.close()

print(demos)

[['BBIJ....IJCC..IAAMGDDK.MGH.KL.GHFFL.'], ['..I...BBI.K.GHAAKLGHDDKLG..JEEFF.J..'], ['JBBCCCJDD..MJAAL.MFFKL.N..KGGN.HH...'], ['BBB..MCCDD.MAAKL.MJ.KLEEJ.GG..JHHHII', 'J0', 'B4'], ['IJBBCCIJDDL.IJAAL.EEK.L...KFF..GGHH.', 'F0', 'G6'], ['BB.G.HE..G.HEAAG.I..FCCIDDF..I..F...']]


# Setting the boards up

In [130]:
boards = []
for data in demos:
  # Seperate Grid Data from the Fuel Parameters
  grid = data[0];
  fuels = []
  if(len(data) > 1):
    fuels.append(data[1:])

  cars = []
  for x in range(6):
    for y in range(6):
      current = grid[x*6 + y];
      
      # If Unique: Create a car and append
      # If Not Unique: Concat to same letter
      if current == ".":
        continue

      unique = True
      for c in cars:
        if c.compareLetters(current):
          c.addCoordinate(Coordinate(x,y))
          unique = False

      if(unique):
        # Now check if there's a unique fuel amount or if its the default 100
        fuelAmt = 100
        if(len(fuels) > 0):
          for f in fuels[0]:
            if current == f[0]:
              fuelAmt = f[1:]

        cars.append(Car.create(current, Coordinate(x,y), fuelAmt));

  # Checking if all cars were made properly
  print("CARS: ")
  print("--------")
  for c in cars:
    print(c)
  print()

  board = Board(cars);
  # Checking if all boards were made properly
  print("BOARD: ")
  print("--------")
  print(board)
  print()

  boards.append(board)

CARS: 
--------
B -> (0, 0)(0, 1) | 100 | Direction.VERTICAL
I -> (0, 2)(1, 2)(2, 2) | 100 | Direction.HORIZONTAL
J -> (0, 3)(1, 3) | 100 | Direction.HORIZONTAL
C -> (1, 4)(1, 5) | 100 | Direction.VERTICAL
A -> (2, 3)(2, 4) | 100 | Direction.VERTICAL
M -> (2, 5)(3, 5) | 100 | Direction.HORIZONTAL
G -> (3, 0)(4, 0)(5, 0) | 100 | Direction.HORIZONTAL
D -> (3, 1)(3, 2) | 100 | Direction.VERTICAL
K -> (3, 3)(4, 3) | 100 | Direction.HORIZONTAL
H -> (4, 1)(5, 1) | 100 | Direction.HORIZONTAL
L -> (4, 4)(5, 4) | 100 | Direction.HORIZONTAL
F -> (5, 2)(5, 3) | 100 | Direction.VERTICAL

BOARD: 
--------
BBIJ..
..IJCC
..IAAM
GDDK.M
GH.KL.
GHFFL.


CARS: 
--------
I -> (0, 2)(1, 2) | 100 | Direction.HORIZONTAL
B -> (1, 0)(1, 1) | 100 | Direction.VERTICAL
K -> (1, 4)(2, 4)(3, 4) | 100 | Direction.HORIZONTAL
G -> (2, 0)(3, 0)(4, 0) | 100 | Direction.HORIZONTAL
H -> (2, 1)(3, 1) | 100 | Direction.HORIZONTAL
A -> (2, 2)(2, 3) | 100 | Direction.VERTICAL
L -> (2, 5)(3, 5) | 100 | Direction.HORIZONTAL
D -

# State Space Search - Uniform Cost Search (UCS)

In [ ]:
# Later